In [228]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
import numpy as np
from torchsummaryX import summary

In [255]:
class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()

        ## Conv Layers
        self.conv_l1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.bn_l1 = nn.BatchNorm2d(16)
            
        self.conv_l2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.bn_l2 = nn.BatchNorm2d(32)
        
        self.conv_l3 = nn.Conv2d(in_channels=32, out_channels=48, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.bn_l3 = nn.BatchNorm2d(48)
        
        self.conv_l4 = nn.Conv2d(in_channels=48, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.bn_l4 = nn.BatchNorm2d(64)
        
        self.conv_l5 = nn.Conv2d(in_channels=64, out_channels=80, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.bn_l5 = nn.BatchNorm2d(80)
        
        self.leakyRelu = torch.nn.LeakyReLU(negative_slope=0.01)
        self.pool = torch.nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
        self.dropout = torch.nn.Dropout(p=0.2)
        
        self.lstm_l1 = torch.nn.LSTM(input_size=1280, hidden_size=256, batch_first=True, bidirectional=True, num_layers=5)
        
        
        
    def forward(self,x):
        
        ## Conv Layers 
        x = self.conv_l1(x)
        x = self.bn_l1(x)
        x = self.leakyRelu(x)
        x = self.pool(x)
        
        x = self.conv_l2(x)
        x = self.bn_l2(x)
        x = self.leakyRelu(x)
        x = self.pool(x)
        
        x = self.dropout(x)
        x = self.conv_l3(x)
        x = self.bn_l3(x)
        x = self.leakyRelu(x)
        x = self.pool(x)
                
        x = self.dropout(x)
        x = self.conv_l4(x)
        x = self.bn_l4(x)
        x = self.leakyRelu(x)
        
        x = self.dropout(x)
        x = self.conv_l5(x)
        x = self.bn_l5(x)
        x = self.leakyRelu(x)
        
        ## LSTM
        x = x.view(1, 128, -1)
        #batch_size, num_channels, input_size = x.shape
        #print (batch_size, num_channels, input_size)
        #print (type(x))
        
        h0 = torch.randn(5*2, 1, 256).to("cuda")
        c0 = torch.randn(5*2, 1, 256).to("cuda")
        
        x,(hn,cn) = self.lstm_l1(x,(h0,c0))
        
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam (self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x,y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

In [256]:
model = LitModel().to("cuda")

In [257]:
summary(model.to("cuda"), torch.zeros(1,1,1024,128).to("cuda"))

                Kernel Shape        Output Shape     Params    Mult-Adds
Layer                                                                   
0_conv_l1      [1, 16, 3, 3]  [1, 16, 1024, 128]      160.0   18.874368M
1_bn_l1                 [16]  [1, 16, 1024, 128]       32.0         16.0
2_leakyRelu                -  [1, 16, 1024, 128]          -            -
3_pool                     -    [1, 16, 512, 64]          -            -
4_conv_l2     [16, 32, 3, 3]    [1, 32, 512, 64]      4.64k  150.994944M
5_bn_l2                 [32]    [1, 32, 512, 64]       64.0         32.0
6_leakyRelu                -    [1, 32, 512, 64]          -            -
7_pool                     -    [1, 32, 256, 32]          -            -
8_dropout                  -    [1, 32, 256, 32]          -            -
9_conv_l3     [32, 48, 3, 3]    [1, 48, 256, 32]    13.872k  113.246208M
10_bn_l3                [48]    [1, 48, 256, 32]       96.0         48.0
11_leakyRelu               -    [1, 48, 256, 32]   

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_conv_l1,"[1, 16, 3, 3]","[1, 16, 1024, 128]",160.0,18874368.0
1_bn_l1,[16],"[1, 16, 1024, 128]",32.0,16.0
2_leakyRelu,-,"[1, 16, 1024, 128]",NaN,NaN
3_pool,-,"[1, 16, 512, 64]",NaN,NaN
4_conv_l2,"[16, 32, 3, 3]","[1, 32, 512, 64]",4640.0,150994944.0
5_bn_l2,[32],"[1, 32, 512, 64]",64.0,32.0
6_leakyRelu,-,"[1, 32, 512, 64]",NaN,NaN
7_pool,-,"[1, 32, 256, 32]",NaN,NaN
8_dropout,-,"[1, 32, 256, 32]",NaN,NaN


In [225]:
inp = torch.rand(1, 1, 1024, 128).to("cuda")

In [226]:
model(inp).shape

1280 1 128


torch.Size([1280, 1, 256])

In [227]:
model.

ModuleAttributeError: 'LitModel' object has no attribute 'summary'